<a href="https://colab.research.google.com/github/EastHuni/lg-aimers/blob/main/LTSF___OPTUNA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 15.7 MB/s eta 0:00:00


In [4]:
from google.colab import drive
import os, glob, random, re, zipfile, json
import pandas as pd
import numpy as np
import holidays
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import optuna

drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/LG/open.zip'
extract_path = '/content/LG_data'
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

train = pd.read_csv('/content/LG_data/train/train.csv')
kr_holidays = holidays.KR(years=range(2023, 2026))

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

class NLinear(nn.Module):
    def __init__(self, input_len=28, output_len=7, input_dim=5, dropout=0.0):
        super(NLinear, self).__init__()
        self.linear = nn.Linear(input_len, output_len)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (B, C, T)
        x = self.dropout(x)
        x = self.linear(x)      # (B, C, output_len)
        return x.permute(0, 2, 1)[:, :, 0]  # (B, output_len)

LOOKBACK, PREDICT = 28, 7 # 이건 고정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def make_features(df):
    df = df.sort_values(['영업장명_메뉴명', '영업일자']).copy()
    df['영업일자'] = pd.to_datetime(df['영업일자'])
    df['요일'] = df['영업일자'].dt.dayofweek
    df['공휴일'] = df['영업일자'].isin(kr_holidays).astype(int)
    df['lag_1'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(1)
    df['rolling_7_mean'] = (
        df.groupby('영업장명_메뉴명')['매출수량']
        .shift(1).rolling(7, min_periods=1).mean().reset_index(0, drop=True))

    feature_cols = ['lag_1', 'rolling_7_mean']
    df[feature_cols] = df.groupby('영업장명_메뉴명')[feature_cols].ffill()
    rolling_avg = (
        df.groupby('영업장명_메뉴명')['매출수량']
        .transform(lambda x: x.shift(1).rolling(7, min_periods=1).mean()))

    for col in feature_cols:
        df[col] = df[col].fillna(rolling_avg)
    df[feature_cols] = df[feature_cols].fillna(0)

    return df[['영업일자', '영업장명_메뉴명', '매출수량', '요일', '공휴일'] + feature_cols]

train = make_features(train)


def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff) * 100


def objective(trial):
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    epochs = trial.suggest_int('epochs', 10, 50)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)

    total_preds, total_trues = [], []

    for store_menu, group in train.groupby('영업장명_메뉴명'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT + 7:
            continue

        features = ['매출수량', '요일', '공휴일', 'lag_1', 'rolling_7_mean']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        values = store_train[features].values

        val_target = values[-7:, 0]
        train_values = values[:-7]

        X_train, y_train = [], []
        for i in range(len(train_values) - LOOKBACK - PREDICT + 1):
            X_train.append(train_values[i:i+LOOKBACK])
            y_train.append(train_values[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        if len(X_train) == 0:
            continue

        X_train = torch.tensor(np.array(X_train)).float().to(DEVICE)
        y_train = torch.tensor(np.array(y_train)).float().to(DEVICE)

        model = NLinear(input_len=LOOKBACK, output_len=PREDICT, input_dim=5, dropout=dropout).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = torch.nn.MSELoss()

        model.train()
        for epoch in range(epochs):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), batch_size):
                batch_idx = idx[i:i+batch_size]
                x_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(x_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        last_seq = train_values[-LOOKBACK:]
        x_val = torch.tensor([last_seq]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_val).squeeze().cpu().numpy()

        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 5))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(restored_val)

        total_preds.extend(restored)
        total_trues.extend(val_target)

    return smape(np.array(total_trues), np.array(total_preds))

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

with open("best_params.json", "w") as f:
    json.dump(study.best_trial.params, f)

print("\n✅ Best SMAPE:", study.best_value)
print("✅ Best Params:", study.best_trial.params)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-4251228950.py:66: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['공휴일'] = df['영업일자'].isin(kr_holidays).astype(int)
[I 2025-08-06 20:02:26,231] A new study created in memory with name: no-name-f97ec754-e835-44ae-9d85-1f571820f361
/tmp/ipython-input-4251228950.py:145: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x_val = torch.tensor([last_seq]).float().to(DEVICE)
[I 2025-08-06 20:07:45,116] Trial 0 finished with value: 187.4406952964939 and parameters: {'lr': 0.00010635797480554176, 'batch_size': 8, 'epochs': 25, 'dropout': 0.02005748735990437}. 


✅ Best SMAPE: 186.06835970333015
✅ Best Params: {'lr': 0.0006806458018098577, 'batch_size': 16, 'epochs': 20, 'dropout': 0.3870664802526545}
